In [2]:
# -*- coding: utf-8 -*-
"""Experimental.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1sakgMfi7rWd_pO6fIB1zYbAVN1TlxzGk

Открываем исходный файл и файл с клиппингом.

*   wav_file_path - путь до исходного файла (без клиппинга), его данные записываются в массив samples
*   wav_file_path1 - путь до файла с клиппингом, его данные записываются в массив samples1
"""

import wave
import numpy as np

def read_wav_as_float(file_path):
    """
    Reads a WAV file and returns its samples as a list of floating-point values.

    Parameters:
        file_path (str): Path to the WAV file.

    Returns:
        list: A list of floating-point samples.
    """
    # Open the WAV file
    with wave.open(file_path, 'rb') as wav_file:
        # Get parameters
        n_channels = wav_file.getnchannels()
        sample_width = wav_file.getsampwidth()
        n_frames = wav_file.getnframes()
        frame_rate = wav_file.getframerate()

        print(f"Channels: {n_channels}, Sample Width: {sample_width}, Frame Rate: {frame_rate}, Frames: {n_frames}")

        # Read frames as bytes
        raw_data = wav_file.readframes(n_frames)

    # Determine the data type based on sample width
    dtype = {1: np.int8, 2: np.int16, 4: np.int32}.get(sample_width)
    if dtype is None:
        raise ValueError(f"Unsupported sample width: {sample_width}")

    # Convert raw bytes to numpy array
    int_data = np.frombuffer(raw_data, dtype=dtype)

    # Normalize to floating-point range [-1.0, 1.0]
    max_val = float(2 ** (8 * sample_width - 1))
    float_data = int_data / max_val

    # Handle multi-channel audio by averaging channels
    if n_channels > 1:
        float_data = float_data.reshape(-1, n_channels).mean(axis=1)

    return float_data.tolist()

# Example usage
wav_file_path = '1.wav'  # Replace with the path to your WAV file
wav_file_path1 = '1c.wav'
samples = read_wav_as_float(wav_file_path)
samples1 = read_wav_as_float(wav_file_path1)

"""Функция для записи массива в файл по пути output_path."""

def write_float_samples_to_wav(samples, sample_rate, output_path):
    """
    Writes floating-point audio samples to a mono 16-bit WAV file.

    Parameters:
        samples (list or np.ndarray): Array of floating-point audio samples in the range [-1.0, 1.0].
        sample_rate (int): Sample rate of the audio in Hz (e.g., 44100).
        output_path (str): Path to save the output WAV file.
    """
    # Ensure the samples are a NumPy array
    samples = np.array(samples, dtype=np.float32)

    # Clip the samples to the range [-1.0, 1.0] to prevent overflow
    samples = np.clip(samples, -1.0, 1.0)

    # Convert to 16-bit PCM format
    int_samples = (samples * 32767).astype(np.int16)

    # Write to a WAV file
    with wave.open(output_path, 'wb') as wav_file:
        # Set the parameters for the WAV file
        wav_file.setnchannels(1)  # Mono
        wav_file.setsampwidth(2)  # 16-bit PCM
        wav_file.setframerate(sample_rate)

        # Write the audio frames
        wav_file.writeframes(int_samples.tobytes())
#import random
#for i in range(100):
#  start = random.randint(0,len(samples1))
#  samples1[start:start+15] = [1e-10]*15
#write_float_samples_to_wav(samples1, 16000, "1c16zeros.wav")

"""Разбиваем оба файла на последовательности по SQNC_LENGTH сэмплов, сэмплы исходного файла в samples_sequences, искаженного в sampeles_sequences_clipped."""

j = 0
SQNC_LENGTH = 256
samples_sequences = []
samples_sequences_clipped = []
while j < len(samples1):
    if(j+SQNC_LENGTH < len(samples1)):
        samples_sequences.append(samples[j:j+SQNC_LENGTH])
        samples_sequences_clipped.append(samples1[j:j+SQNC_LENGTH])
    j += SQNC_LENGTH

"""Обучение нейросети на множестве спектрограмм сигнала. N и M - количество точек по осям частоты и времени соответственно в обучающих выборках."""

import tensorflow as tf

# Custom STFT layer using tf.signal.stft
class STFTLayer(tf.keras.layers.Layer):
    def __init__(self, frame_length=8, frame_step=4, **kwargs):
        super(STFTLayer, self).__init__(**kwargs)
        self.frame_length = frame_length
        self.frame_step = frame_step

    def call(self, inputs):
        # inputs: shape (batch, sq_lngth)
        # Use a Hann window
        window = tf.signal.hann_window(self.frame_length, dtype=inputs.dtype)
        stft_result = tf.signal.stft(
            inputs,
            frame_length=self.frame_length,
            frame_step=self.frame_step,
            window_fn=lambda fl, dtype: window
        )
        # tf.signal.stft returns shape (batch, time_frames, fft_unique_bins)
        # For our design, we want to use (batch, fft_unique_bins, time_frames)
        magnitude = tf.abs(stft_result)
        phase = tf.math.angle(stft_result)
        # Transpose to shape (batch, fft_unique_bins, time_frames)
        magnitude = tf.transpose(magnitude, perm=[0, 2, 1])
        phase = tf.transpose(phase, perm=[0, 2, 1])
        return magnitude, phase

    def compute_output_shape(self, input_shape):
        batch = input_shape[0]
        if input_shape[1] is None:
            return (batch, None, None), (batch, None, None)
        # time_frames computed from signal length:
        time_frames = (input_shape[1] - self.frame_length) // self.frame_step + 1
        fft_bins = self.frame_length // 2 + 1
        # After transposition, output shape becomes (batch, fft_bins, time_frames)
        return (batch, fft_bins, time_frames), (batch, fft_bins, time_frames)


# Custom inverse STFT layer using tf.signal.inverse_stft
class ISTFTLayer(tf.keras.layers.Layer):
    def __init__(self, frame_length=8, frame_step=4, sq_lngth=None, **kwargs):
        super(ISTFTLayer, self).__init__(**kwargs)
        self.frame_length = frame_length
        self.frame_step = frame_step
        self.sq_lngth = sq_lngth

    def call(self, inputs):
        # inputs: a list [mag, phase] with shapes (batch, F, T)
        mag, phase = inputs
        # tf.signal.inverse_stft expects input of shape (batch, time_frames, fft_unique_bins).
        # So transpose mag and phase from (batch, F, T) to (batch, T, F):
        mag_t = tf.transpose(mag, perm=[0, 2, 1])
        phase_t = tf.transpose(phase, perm=[0, 2, 1])
        stft_complex = tf.cast(mag_t, tf.complex64) * tf.exp(1j * tf.cast(phase_t, tf.complex64))
        window = tf.signal.hann_window(self.frame_length, dtype=tf.float32)
        reconstructed = tf.signal.inverse_stft(
            stft_complex,
            frame_length=self.frame_length,
            frame_step=self.frame_step,
            window_fn=lambda fl, dtype: window
        )
        if self.sq_lngth is not None:
            reconstructed = reconstructed[:, :self.sq_lngth]
        return reconstructed

    def compute_output_shape(self, input_shape):
        batch = input_shape[0][0]
        if self.sq_lngth is not None:
            return (batch, self.sq_lngth)
        else:
            return (batch, None)


# Helper layers to add and remove a singleton channel dimension.
class AddInnerDim(tf.keras.layers.Layer):
    def call(self, x):
        return tf.expand_dims(x, axis=-1)

class Squeeze(tf.keras.layers.Layer):
    def call(self, x):
        return tf.squeeze(x, axis=-1)


# Updated model-building function using differentiable STFT/ISTFT
def build_rnn_spectrogram_model(sq_lngth):
    # Define input: a 1D signal of length sq_lngth (batch dimension preserved)
    input_tensor = tf.keras.layers.Input(shape=(sq_lngth,))  # shape: (batch, sq_lngth)

    # Compute the STFT using our custom layer.
    stft_layer = STFTLayer(frame_length=8, frame_step=4)
    mag, phase = stft_layer(input_tensor)
    # Now mag and phase have shape (batch, F, T) where F = 5 and T = (sq_lngth-8)//4 + 1.

    # For consistency, set T = M_const:
    M_const = (sq_lngth - 8) // 4 + 1
    mag = mag[:, :, :M_const]   # shape: (batch, 5, M_const)
    phase = phase[:, :, :M_const]  # shape: (batch, 5, M_const)

    # Add a channel dimension (for Conv2D) while preserving the batch size.
    mag = AddInnerDim()(mag)      # shape: (batch, 5, M_const, 1)
    phase = AddInnerDim()(phase)  # shape: (batch, 5, M_const, 1)

    # Process magnitude with 2D convolutions.
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(mag)
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    # Now x has shape (batch, 5, M_const, 64)

    # Reshape for RNN processing.
    # We treat the frequency dimension (5) as timesteps, and flatten the remaining dims.
    x = tf.keras.layers.Reshape((5, M_const * 64))(x)  # shape: (batch, 5, M_const*64)

    # Process with two SimpleRNN layers.
    x = tf.keras.layers.SimpleRNN(units=256, activation='relu', return_sequences=True)(x)
    '''x = tf.keras.layers.SimpleRNN(units=128, activation='relu', return_sequences=True)(x)
    x = tf.keras.layers.SimpleRNN(units=64, activation='relu', return_sequences=True)(x)
    x = tf.keras.layers.SimpleRNN(units=32, activation='relu', return_sequences=True)(x)'''
    # Map each of the 5 timesteps to M_const outputs via a Dense layer.
    x = tf.keras.layers.Dense(units=M_const, activation='linear')(x)  # shape: (batch, 5, M_const)

    # x now represents the processed magnitude spectrogram (shape: (batch, 5, M_const)).
    # For phase, remove the extra channel dimension.
    phase = Squeeze()(phase)  # shape: (batch, 5, M_const)

    # Use the ISTFT layer to reconstruct the time-domain signal.
    istft_layer = ISTFTLayer(frame_length=8, frame_step=4, sq_lngth=sq_lngth)
    output_signal = istft_layer([x, phase])  # shape: (batch, sq_lngth)

    model = tf.keras.models.Model(inputs=input_tensor, outputs=output_signal)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_absolute_error',
        metrics=['mse']
    )
    return model


# Example usage:
# Assume SQNC_LENGTH, samples_sequences_clipped, and samples_sequences are defined.
model = build_rnn_spectrogram_model(SQNC_LENGTH)
model.summary()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)
model.fit(np.array(samples_sequences_clipped), np.array(samples_sequences),
          batch_size=32, epochs=1000, callbacks=[early_stopping])

vect = np.random.rand(SQNC_LENGTH)
vect = np.expand_dims(vect, axis=0)  # Now shape is (1, SQNC_LENGTH)
print(model.predict(vect))

"""Открытие файла который нужно восстановить и получение массива его спектрограмм. file_for_restoration_path - путь к файлу который нужно восстановить.
samples_input_sequences - массив семплов этого файла

Zyy,phsy - массивы амплитудных и фазовых спектрограмм файла соответственно
"""

file_for_restoration_path = "1c.wav"
samples_input_file = read_wav_as_float(file_for_restoration_path)
j = 0
SQNC_LENGTH = 256
fs = 44100
samples_input_sequences = []
while j < len(samples_input_file ):
    if(j+SQNC_LENGTH < len(samples_input_file)):
        samples_input_sequences.append(samples_input_file[j:j+SQNC_LENGTH])
    j += SQNC_LENGTH
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.signal import stft, istft
import wave
# Open the WAV file
with wave.open(file_for_restoration_path, 'rb') as wav_file:
    fs = wav_file.getframerate()
Zyy = []
phsy = []
t = np.linspace(0, 1.0, SQNC_LENGTH, endpoint=False)  # Time vector
for i in range(len(samples_input_sequences)):
  signal = samples_input_sequences[i]
  # Parameters for STFT

  #shape of the result is (a,b) where a = frame_length//2+1 and b = ceil(N//a), N - number of samples in original sequence
  frame_length = 8  # Frame length (number of samples per frame)
  frame_step = frame_length//2  # Step between frames (overlap)

  # Compute the STFT using scipy's stft function
  f, t_stft, Z = stft(signal, fs, nperseg=frame_length, noverlap=frame_step)
  Zyy.append(np.abs(Z))
  phsy.append(tf.math.angle(Z))


Channels: 1, Sample Width: 2, Frame Rate: 44100, Frames: 122880
Channels: 1, Sample Width: 2, Frame Rate: 44100, Frames: 122880


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 256)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stft_layer_1 (STFTLayer)  │ [(None, 5, 63), (None, │              0 │ input_layer_1[0][0]    │
│                           │ 5, 63)]                │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_2 (GetItem)      │ (None, 5, 63)          │              0 │ stft_layer_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_inner_dim_2           │ (None, 5, 63, 1)       │              0 │ get_item_2[0][0]       │
│ (AddInnerDim)             │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 5, 63, 64)      │            640 │ add_inner_dim_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 5, 63, 64)      │         36,928 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_1 (Reshape)       │ (None, 5, 4032)        │              0 │ conv2d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_3 (GetItem)      │ (None, 5, 63)          │              0 │ stft_layer_1[0][1]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ simple_rnn_1 (SimpleRNN)  │ (None, 5, 256)         │      1,097,984 │ reshape_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_inner_dim_3           │ (None, 5, 63, 1)       │              0 │ get_item_3[0][0]       │
│ (AddInnerDim)             │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 5, 63)          │         16,191 │ simple_rnn_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ squeeze_1 (Squeeze)       │ (None, 5, 63)          │              0 │ add_inner_dim_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ istft_layer_1             │ (None, 256)            │              0 │ dense_1[0][0],         │
│ (ISTFTLayer)              │                        │                │ squeeze_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,151,743 (4.39 MB)

 Trainable params: 1,151,743 (4.39 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000


15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - loss: 0.0459 - mse: 0.0091
Epoch 2/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 0.0327 - mse: 0.0048
Epoch 3/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0222 - mse: 0.0025
Epoch 4/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - loss: 0.0188 - mse: 0.0020
Epoch 5/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0158 - mse: 0.0016
Epoch 6/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0140 - mse: 0.0014
Epoch 7/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0135 - mse: 0.0012
Epoch 8/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 0.0124 - mse: 0.0012
Epoch 9/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0134 - mse: 0.0014
Epoch 10/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0134 - mse: 0.0013
Epoch 11/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0109 - mse: 0.0011
Epoch 12/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0111 - mse: 0.0012
Epoch 13/1000
15/15 ━━━━━━

In [ ]:
"""Восстановление звука с помощью нейросети. samples_restored - массив восстановленных сэмплов звука."""

import cmath
import math
samples_restored = []
maxv = np.max(np.array(samples_input_sequences))
minv = np.min(np.array(samples_input_sequences))
for i in range(len(samples_input_sequences)):
  t = np.linspace(0, 1.0, SQNC_LENGTH, endpoint=False)  # Time vector
  #shape of the result is (a,b) where a = frame_length//2+1 and b = ceil(N//a), N - number of samples in original sequence
  if(len(samples_input_sequences[i])<SQNC_LENGTH):
    break
  if(max(samples_input_sequences[i])>(maxv*0.95) or min(samples_input_sequences[i])<(minv*0.95)):
    vect = np.array(samples_input_sequences[i])
    vect = np.expand_dims(vect, axis=0)
    reconstructed = model.predict(vect,verbose=0)
    samples_restored.append(reconstructed[0:SQNC_LENGTH])
  else:
    samples_restored.append(np.array(samples_input_sequences[i]))

In [ ]:
"""Если мы хотим произвести сравнение с каким-либо другим методом, возможно, возникнет проблема из-за разных длин файлов: текущий алгоритм отбрасывает последние сэмплы в файле чтобы достичь количества сэмплов кратного SQNC_LENGTH. Если раскомментировать вторую строку мы получим массив в котором недостающие восстановленные сэмплы заменены сэмплами исходного массива до требуемой длины, что обеспечит возможность сравнения файлов. output_path - название файла, в который будет записан вывод программы."""

samples_restored_final = samples_restored
for i in range(len(samples_restored_final)):
  print(type(samples_restored_final),len(samples_restored_final[i]))
samples_restored_final = np.append(np.array(samples_restored_final).flatten(),np.array(samples_input_file[len(samples_input_file)-(len(samples_input_file) % SQNC_LENGTH)::]))
import wave
import numpy as np

def write_float_samples_to_wav(samples, sample_rate, output_path):
    """
    Writes floating-point audio samples to a mono 16-bit WAV file.

    Parameters:
        samples (list or np.ndarray): Array of floating-point audio samples in the range [-1.0, 1.0].
        sample_rate (int): Sample rate of the audio in Hz (e.g., 44100).
        output_path (str): Path to save the output WAV file.
    """
    # Ensure the samples are a NumPy array
    samples = np.array(samples, dtype=np.float32)

    # Clip the samples to the range [-1.0, 1.0] to prevent overflow
    samples = np.clip(samples, -1.0, 1.0)

    # Convert to 16-bit PCM format
    int_samples = (samples * 32767).astype(np.int16)

    # Write to a WAV file
    with wave.open(output_path, 'wb') as wav_file:
        # Set the parameters for the WAV file
        wav_file.setnchannels(1)  # Mono
        wav_file.setsampwidth(2)  # 16-bit PCM
        wav_file.setframerate(sample_rate)

        # Write the audio frames
        wav_file.writeframes(int_samples.tobytes())

output_path = 'output.wav'  # Path to save the WAV file

write_float_samples_to_wav(samples_restored_final, fs, output_path)
print(f"WAV file written to {output_path}")

In [ ]:
"""Занимательные картинки."""

import cmath
import math
for i in range(20,len(samples_input_sequences)):
  t = np.linspace(0, 1.0, SQNC_LENGTH, endpoint=False)  # Time vector
  plt.subplot(2, 1, 2)

  plt.plot(t, samples_sequences[i], label="Original Signal", color='blue')
  plt.plot(t, samples_input_sequences[i], label="Clipped Signal", color='green')
  plt.plot(t, samples_restored[i].flatten(), label="Reconstructed Signal", color='orange')
  plt.title('Reconstructed Signal from Spectrogram')
  plt.xlabel('Time [s]')
  plt.ylabel('Amplitude')
  plt.legend()

  plt.tight_layout()
  plt.show()